In [4]:
import os
import luigi
import io
import pandas as pd
import numpy as np
import logging
from emu.pipeline.download import FileManifest,PatientsLocal,CacheTaskOutput,ExperimentManifest
from emu.pipeline.remote import RemotePatientManifest,RemoteCSV,Patients
# from emu.pipeline.utils import file_ids_by_channel
from emu.luigi.box import BoxTarget, path_to_fid
from emu.pipeline.process import PDilTask
from emu.utils import get_file_manifest,load_patients,Experiment
from emu.auth import jwt
from emu.pdil.raw import BehaviorRaw, timestamps,extract_trial_timing
# from emu.pdil.raw import PDilCache as PDC
from emu.pdil.raw import Participant,get_data_manifest

In [5]:
root_dir = os.path.expanduser('~/.emu/')

In [6]:
data_manifest = get_data_manifest()

In [7]:
data_manifest

,filename,folder,id,type,patient_id
0,CSC100_0005.ncs,PO_Day_02,632794409407,SEEG,1
1,CSC100_0006.ncs,PO_Day_02,632794401007,SEEG,1
2,CSC100_0007.ncs,PO_Day_02,632794395007,SEEG,1
3,CSC101_0005.ncs,PO_Day_02,632794407007,SEEG,1
4,CSC101_0006.ncs,PO_Day_02,632794410607,SEEG,1
...,...,...,...,...,...
763,Survey_postBlock2.jpeg,PT01_postOpDay2_PDil,630717007369,NaN,1
768,pt01_round1_NOreveal.asc,PT01_postOpDay4_trickshots,629612753923,eyetracking,1
769,pt01_round1_NOreveal.edf,PT01_postOpDay4_trickshots,629631897388,eyetracking,1
770,pt01_round2_WITHreveal.asc,PT01_postOpDay4_trickshots,629631988086,eyetracking,1


In [9]:
data_manifest.groupby(['type','folder']).count()

filename   id  patient_id
type        folder                                               
Behavior    PT01_postOpDay2_PDil               3    3           3
            PT01_postOpDay4_PDil               4    4           4
SEEG        PO_Day_02                        551  551         551
            PO_Day_04                        196  196         196
            PT_01                              1    1           1
eyetracking PT01_postOpDay4_trickshots         4    4           4
survey      PT_01                              2    2           2

In [14]:
behavior_files = data_manifest.query('patient_id == 1 & type == "Behavior"')
behavior_files

,filename,id,type,patient_id
70,blockNum_3_computerTT_defect_taskoutput.mat,629615908637,Behavior,1
71,blockNum_4_humanTT_defect_taskoutput.mat,629620274728,Behavior,1
72,blockNum_5_humanTT_coop_taskoutput.mat,629611713944,Behavior,1
73,blockNum_6_computerTT_coop_taskoutput.mat,629611688205,Behavior,1
78,blockNum_1_computerTT_coop_taskoutput.mat,628721484958,Behavior,1
79,blockNum_2_humanTT_coop_taskoutput.mat,628728186668,Behavior,1
80,PRACTICE_blockNum_0 _taskoutput.mat,628698648907,Behavior,1


In [16]:
p = Participant(patient_id=1, raw_files=behavior_files, seeg_raw_path='/home/elijahc/.emu/pdil/pt_01/SEEG/raw/')

In [18]:
game = pd.concat(p.load_game_data())

DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629615908637, file_name=blockNum_3_computerTT_defect_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   BehaviorRaw__home_elijahc__e_629615908637_blockNum_3_compu_3484a4f2e7   has status   DONE
DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629620274728, file_name=blockNum_4_humanTT_defect_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   BehaviorRaw__home_elijahc__e_629620274728_blockNum_4_human_73d59337a6   has status   DONE
DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629611713944, file_name=blockNum_5_humanTT_coop_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed s

In [4]:
raw_files = pdc.data_manifest.query('patient_id == 1')

In [5]:
seeg_raw = os.path.join('/home/elijahc/.emu/pdil/pt_01/SEEG/raw')
p = Participant(patient_id=1,raw_files=raw_files,seeg_raw_path=seeg_raw)

In [6]:
games = p.load_game_data()
next(games)

DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629615908637, file_name=blockNum_3_computerTT_defect_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   BehaviorRaw__home_elijahc__e_629615908637_blockNum_3_compu_3484a4f2e7   has status   DONE
DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629620274728, file_name=blockNum_4_humanTT_defect_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   BehaviorRaw__home_elijahc__e_629620274728_blockNum_4_human_73d59337a6   has status   DONE
DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629611713944, file_name=blockNum_5_humanTT_coop_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed s

,points,reaction_time,timing_sumTictocs,timing_wholesession,trial,block,opponent,strategy
0,4,1.887549,16.915350,16.937015,1,3,computer,defect
1,4,1.336412,14.620682,14.638466,2,3,computer,defect
2,6,1.206812,14.253415,14.264153,3,3,computer,defect
3,1,1.016557,14.099830,14.112599,4,3,computer,defect
4,6,3.820575,34.350554,34.375759,5,3,computer,defect
5,1,4.389647,20.046147,20.071950,6,3,computer,defect
6,1,1.622268,14.245598,14.263381,7,3,computer,defect
7,2,1.144656,14.924340,14.927988,8,3,computer,defect
8,2,1.290998,14.659205,14.671975,9,3,computer,defect
9,4,0.885510,32.880432,32.903222,10,3,computer,defect


In [21]:
p_events = pd.concat(p.load_pdil_events())

In [28]:
import emu.neuralynx_io as nlx

In [31]:
seeg_root = '/home/elijahc/.emu/pdil/pt_01/SEEG/raw'
ev = nlx.load_nev(os.path.join(seeg_root,'Events.nev'))

/home/elijahc/dev/emu/emu/neuralynx_io.py:79: UserWarning: Unable to parse original file path from Neuralynx header: -FileType Event
  warnings.warn('Unable to parse original file path from Neuralynx header: ' + hdr_lines[1])
/home/elijahc/dev/emu/emu/neuralynx_io.py:131: UserWarning: Unable to parse time string from Neuralynx header: -FileVersion 3.2
  warnings.warn('Unable to parse time string from Neuralynx header: ' + time_string)
/home/elijahc/dev/emu/emu/neuralynx_io.py:131: UserWarning: Unable to parse time string from Neuralynx header: -FileUUID 38d9f340-1658-4df2-abc6-38b9f084701d
  warnings.warn('Unable to parse time string from Neuralynx header: ' + time_string)
/home/elijahc/dev/emu/emu/neuralynx_io.py:93: UserWarning: Unable to parse parameter line from Neuralynx header: -TimeCreated 2020/03/06 12:34:58
  warnings.warn('Unable to parse parameter line from Neuralynx header: ' + line)
/home/elijahc/dev/emu/emu/neuralynx_io.py:93: UserWarning: Unable to parse parameter line f

In [33]:
from emu.pdil.raw import nev_as_records

In [38]:
ev = pd.DataFrame.from_records(nev_as_records(nlx.load_nev(os.path.join(seeg_root,'Events.nev'))))
ev[ev.ttl==1]

,EventString,Extra,TimeStamp,event_id,pkt_id,ttl
7,b'ttl',"[0, 0, 0, 0, 0, 0, 0, 0]",1583498419790693,20,0,1
9,b'ttl',"[0, 0, 0, 0, 0, 0, 0, 0]",1583498420007380,20,0,1
11,b'ttl',"[0, 0, 0, 0, 0, 0, 0, 0]",1583498439290310,20,0,1
13,b'ttl',"[0, 0, 0, 0, 0, 0, 0, 0]",1583498456223303,20,0,1
15,b'ttl',"[0, 0, 0, 0, 0, 0, 0, 0]",1583498470856329,20,0,1
17,b'ttl',"[0, 0, 0, 0, 0, 0, 0, 0]",1583498485122729,20,0,1
19,b'ttl',"[0, 0, 0, 0, 0, 0, 0, 0]",1583498499239130,20,0,1
21,b'ttl',"[0, 0, 0, 0, 0, 0, 0, 0]",1583498533621803,20,0,1
23,b'ttl',"[0, 0, 0, 0, 0, 0, 0, 0]",1583498553688108,20,0,1
25,b'ttl',"[0, 0, 0, 0, 0, 0, 0, 0]",1583498567937821,20,0,1


In [ ]:
p.

In [7]:
list(p.cache_behavior())

[BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629615908637, file_name=blockNum_3_computerTT_defect_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1),
 BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629620274728, file_name=blockNum_4_humanTT_defect_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1),
 BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629611713944, file_name=blockNum_5_humanTT_coop_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1),
 BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629611688205, file_name=blockNum_6_computerTT_coop_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1),
 BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=628721484958, file_name=blockNum_1_computerTT_coop_taskoutput.mat, save_to=/home/elijahc/.

In [8]:
pdil = pd.concat(p.load_pdil_events())
pdil.groupby(['block','trial']).count()

event  event_delta  screen
block trial                            
0     0          1            1       0
      1         10           10      10
      2         10           10      10
      3         10           10      10
      4         10           10      10
      5         10           10      10
      6         10           10      10
      7         10           10      10
      8         10           10      10
1     0          1            1       0
      1         10           10      10
      2         10           10      10
      3         10           10      10
      4         10           10      10
      5         10           10      10
      6         10           10      10
      7         10           10      10
      8         10           10      10
      9         10           10      10
      10        10           10      10
      11        10           10      10
      12        10           10      10
      13        10           10      10
      14        10           10      10
      15        10           10      10
2     0          1            1       0
      1         10           10      10
      2         10           10      10
      3         10           10      10
      4         10           10      10
...            ...          ...     ...
5     2         10           10      10
      3         10           10      10
      4         10           10      10
      5         10           10      10
      6         10           10      10
      7         10           10      10
      8         10           10      10
      9         10           10      10
      10        10           10      10
      11        10           10      10
      12        10           10      10
      13        10           10      10
      14        10           10      10
      15        10           10      10
6     0          1            1       0
      1         10           10      10
      2         10           10      10
      3         10           10      10
      4         10           10      10
      5         10           10      10
      6         10           10      10
      7         10           10      10
      8         10           10      10
      9         10           10      10
      10        10           10      10
      11        10           10      10
      12        10           10      10
      13        10           10      10
      14        10           10      10
      15        10           10      10

[105 rows x 3 columns]

In [ ]:
fp = '/home/elijahc/.emu/pdil/pt_01/Behavior/raw/blockNum_1_computerTT_coop_taskoutput.mat'
b1 = sio.loadmat(fp,squeeze_me=True,struct_as_record=False)

In [ ]:
to = b1['taskoutput']

In [ ]:
timing = b1['taskoutput'].timing

In [ ]:
timing.__dict__.keys()

In [ ]:
intro = [
    'Time_scrn_flip_intro1',
    'Time_postscrn_flip_intro1',
    'Time_scrn_flip_intro2',
    'Time_postscrn_flip_intro2',
    'Time_scrn_flip_intro3',
    'Time_postscrn_flip_intro3',
]

trial = [
    'Time_scrn_flip_trials1',
    'Time_postscrn_flip_trials1',
    'Time_scrn_flip_trials2',
    'Time_postscrn_flip_trials2',
    'Time_scrn_flip_trials3',
    'Time_postscrn_flip_trials3',
    'Time_scrn_flip_trials4',
    'Time_postscrn_flip_trials4',
    'Time_scrn_flip_trials5',
    'Time_postscrn_flip_trials5',
]

In [ ]:
num_rounds = 15
records = {
    'delta_t':[],
    'screen':[],
    'event':[],
    'round':[],
    'round_idx':[],
}

for i in np.arange(num_rounds):
    ri = 0
    for t in trial:
        ri+=1
        row = getattr(timing,t)
        records['delta_t'].append(row[i:i+1].max())
        records['round_idx'].append(ri)
        records['screen'].append(t[-1])
        if 'postscrn' in t:
            records['event'].append('keypress{}'.format(t[-1]))
        else:
            records['event'].append(t)
#         records['keypress'].append()
    records['round'].extend([i+1]*len(trial))
        

df = pd.DataFrame.from_records(records).sort_values(['round','screen'])
df['ttl_delta'] = df.delta_t.cumsum()
df.head(11)

In [ ]:
df = extract_trial_timing(fp)
df['ttl_delta'] = df.event_delta.cumsum()
df

In [ ]:
l = lambda s,j: s[:j].delta_t.sum()

In [ ]:
df.groupby('round').apply(l,8)

In [ ]:
getattr(timing,trial[7])

In [ ]:
game = pdc.load_behavior(patient_id=[1])
# game.head(20)